In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

AUDITORIA MUNICIPAL 

In [4]:
auditoria_df = (spark.read.option("delimiter", ";") 
                          .option("header", True)
                          .option('encoding', 'ISO-8859-1') 
                          .csv("s3://bronze/auditoria_municipal/6-siap-net-orgaos-municipais-autoridades-2016.csv"))

24/01/20 13:16:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
auditoria_df.show()

+---------------+--------------------+---------------+--------------------+------------+--------------+--------------------+----------------+-----+---------------+--------------------+-------------------+--------------------+----+-------------+----------+
|CodigoMunicipio|       NomeMunicipio|CodigoTipoOrgao|       NomeTipoOrgao|AnoExercicio|SequenciaOrgao|           NomeOrgao|CodigoAutoridade|Trata|mentoAutoridade|     CargoAutoridade|SequenciaAutoridade|                Nome|Sexo|InicioMandato|FimMandato|
+---------------+--------------------+---------------+--------------------+------------+--------------+--------------------+----------------+-----+---------------+--------------------+-------------------+--------------------+----+-------------+----------+
|---------------|-----------------...|---------------|-----------------...|------------|--------------|-----------------...|----------------|-----|---------------|-----------------...|-------------------|-----------------...|----|--

In [6]:
auditoria_df.printSchema()

root
 |-- CodigoMunicipio: string (nullable = true)
 |-- NomeMunicipio: string (nullable = true)
 |-- CodigoTipoOrgao: string (nullable = true)
 |-- NomeTipoOrgao: string (nullable = true)
 |-- AnoExercicio: string (nullable = true)
 |-- SequenciaOrgao: string (nullable = true)
 |-- NomeOrgao: string (nullable = true)
 |-- CodigoAutoridade: string (nullable = true)
 |-- Trata: string (nullable = true)
 |-- mentoAutoridade: string (nullable = true)
 |-- CargoAutoridade: string (nullable = true)
 |-- SequenciaAutoridade: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- InicioMandato: string (nullable = true)
 |-- FimMandato: string (nullable = true)



In [7]:
from pyspark.sql import functions as F

In [8]:
auditoria_df_01 = auditoria_df.select(F.col('CodigoMunicipio').cast('int')  
                                     ,F.col('NomeMunicipio')
                                     ,F.col('CodigoTipoOrgao').cast('int')   
                                     ,F.col('NomeTipoOrgao') 
                                     ,F.col('AnoExercicio').cast('int') 
                                     ,F.col('SequenciaOrgao').cast('int') 
                                     ,F.col('NomeOrgao') 
                                     ,F.col('CodigoAutoridade').cast('int')
                                     ,F.col('Trata') 
                                     ,F.col('mentoAutoridade') 
                                     ,F.col('CargoAutoridade') 
                                     ,F.col('SequenciaAutoridade').cast('int') 
                                     ,F.col('Nome') 
                                     ,F.col('Sexo') 
                                     ,F.to_date('InicioMandato', 'dd/MM/yyyy').alias('InicioMandato')
                                     ,F.to_date('FimMandato', 'dd/MM/yyyy').alias('FimMandato'))        

In [9]:
auditoria_df_01.printSchema()

root
 |-- CodigoMunicipio: integer (nullable = true)
 |-- NomeMunicipio: string (nullable = true)
 |-- CodigoTipoOrgao: integer (nullable = true)
 |-- NomeTipoOrgao: string (nullable = true)
 |-- AnoExercicio: integer (nullable = true)
 |-- SequenciaOrgao: integer (nullable = true)
 |-- NomeOrgao: string (nullable = true)
 |-- CodigoAutoridade: integer (nullable = true)
 |-- Trata: string (nullable = true)
 |-- mentoAutoridade: string (nullable = true)
 |-- CargoAutoridade: string (nullable = true)
 |-- SequenciaAutoridade: integer (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- InicioMandato: date (nullable = true)
 |-- FimMandato: date (nullable = true)



In [10]:
auditoria_df_01.write.parquet(path= "s3://silver/auditoria_municipal/", mode="overwrite", partitionBy="AnoExercicio")

DESPESAS

In [11]:
despesas_df = (spark.read.option("delimiter", ";") 
                         .option("header", True) 
                         .option('encoding', 'ISO-8859-1') 
                         .csv("s3://bronze/despesas/despesas-2023.csv"))

In [12]:
despesas_df.show()

+------------------+-------------+------------+---------------------+--------------------+--------------+---------------+----------+----------+------------------------+------------------------+--------------------+------------------+----------+------------------+--------------------+-----------+--------------------+-------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
|id_despesa_detalhe|ano_exercicio|ds_municipio|codigo_municipio_ibge|            ds_orgao|mes_referencia|mes_ref_extenso|tp_despesa|nr_empenho|tp_identificador_despesa|nr_identificador_despesa|          ds_despesa|dt_emissao_despesa|vl_despesa| ds_funcao_governo|ds_subfuncao_governo|cd_programa|         ds_programa|cd_acao|             ds_acao|ds_fonte_recurso|ds_cd_aplicacao_fixo|   ds_modalidade_lic|         ds_elemento|   historico_despesa|
+------------------+-------------+------------+---------------------+--------------------+--------------

In [13]:
despesas_df.printSchema()

root
 |-- id_despesa_detalhe: string (nullable = true)
 |-- ano_exercicio: string (nullable = true)
 |-- ds_municipio: string (nullable = true)
 |-- codigo_municipio_ibge: string (nullable = true)
 |-- ds_orgao: string (nullable = true)
 |-- mes_referencia: string (nullable = true)
 |-- mes_ref_extenso: string (nullable = true)
 |-- tp_despesa: string (nullable = true)
 |-- nr_empenho: string (nullable = true)
 |-- tp_identificador_despesa: string (nullable = true)
 |-- nr_identificador_despesa: string (nullable = true)
 |-- ds_despesa: string (nullable = true)
 |-- dt_emissao_despesa: string (nullable = true)
 |-- vl_despesa: string (nullable = true)
 |-- ds_funcao_governo: string (nullable = true)
 |-- ds_subfuncao_governo: string (nullable = true)
 |-- cd_programa: string (nullable = true)
 |-- ds_programa: string (nullable = true)
 |-- cd_acao: string (nullable = true)
 |-- ds_acao: string (nullable = true)
 |-- ds_fonte_recurso: string (nullable = true)
 |-- ds_cd_aplicacao_fixo: 

In [34]:
despesas_df_01 = despesas_df.select(F.col('id_despesa_detalhe')  
                                   ,F.col('ano_exercicio').cast('int') 
                                   ,F.col('ds_municipio')  
                                   ,F.col('codigo_municipio_ibge').cast('int')  
                                   ,F.col('ds_orgao')
                                   ,F.col('mes_referencia').cast('int') 
                                   ,F.col('mes_ref_extenso') 
                                   ,F.col('tp_despesa')
                                   ,F.col('nr_empenho') 
                                   ,F.col('tp_identificador_despesa') 
                                   ,F.col('nr_identificador_despesa').cast('bigint') 
                                   ,F.col('ds_despesa')
                                   ,F.to_date('dt_emissao_despesa', 'dd/MM/yyyy').alias('dt_emissao_despesa')
                                   ,F.regexp_replace(F.col('vl_despesa'), ",", ".").cast('decimal(9,2)').alias('vl_despesa') 
                                   ,F.col('ds_funcao_governo')       
                                   ,F.col('ds_subfuncao_governo')       
                                   ,F.col('cd_programa').cast('int')        
                                   ,F.col('ds_programa')       
                                   ,F.col('cd_acao').cast('int')  
                                   ,F.col('ds_fonte_recurso')        
                                   ,F.col('ds_cd_aplicacao_fixo')        
                                   ,F.col('ds_modalidade_lic')        
                                   ,F.col('ds_elemento')        
                                   ,F.col('historico_despesa'))

In [35]:
despesas_df_01.printSchema()

root
 |-- id_despesa_detalhe: string (nullable = true)
 |-- ano_exercicio: integer (nullable = true)
 |-- ds_municipio: string (nullable = true)
 |-- codigo_municipio_ibge: integer (nullable = true)
 |-- ds_orgao: string (nullable = true)
 |-- mes_referencia: integer (nullable = true)
 |-- mes_ref_extenso: string (nullable = true)
 |-- tp_despesa: string (nullable = true)
 |-- nr_empenho: string (nullable = true)
 |-- tp_identificador_despesa: string (nullable = true)
 |-- nr_identificador_despesa: long (nullable = true)
 |-- ds_despesa: string (nullable = true)
 |-- dt_emissao_despesa: date (nullable = true)
 |-- vl_despesa: decimal(9,2) (nullable = true)
 |-- ds_funcao_governo: string (nullable = true)
 |-- ds_subfuncao_governo: string (nullable = true)
 |-- cd_programa: integer (nullable = true)
 |-- ds_programa: string (nullable = true)
 |-- cd_acao: integer (nullable = true)
 |-- ds_fonte_recurso: string (nullable = true)
 |-- ds_cd_aplicacao_fixo: string (nullable = true)
 |-- ds

In [36]:
despesas_df_01.show()

+------------------+-------------+------------+---------------------+--------------------+--------------+---------------+----------+----------+------------------------+------------------------+--------------------+------------------+----------+------------------+--------------------+-----------+--------------------+-------+----------------+--------------------+--------------------+--------------------+--------------------+
|id_despesa_detalhe|ano_exercicio|ds_municipio|codigo_municipio_ibge|            ds_orgao|mes_referencia|mes_ref_extenso|tp_despesa|nr_empenho|tp_identificador_despesa|nr_identificador_despesa|          ds_despesa|dt_emissao_despesa|vl_despesa| ds_funcao_governo|ds_subfuncao_governo|cd_programa|         ds_programa|cd_acao|ds_fonte_recurso|ds_cd_aplicacao_fixo|   ds_modalidade_lic|         ds_elemento|   historico_despesa|
+------------------+-------------+------------+---------------------+--------------------+--------------+---------------+----------+----------+---

In [38]:
despesas_df_01.write.parquet(path= "s3://silver/despesas/", mode="overwrite", partitionBy="ano_exercicio")